This notebook serves to get additional data from the EDH XML/epidoc files. The reason is that some information is missing in the API data.


In [2]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import sddk

In [ ]:
### TO BE IMPLEMENTED:

# to make a loop to get all zip files from https://edh-www.adw.uni-heidelberg.de/data/export and to parse the data using beautiful soup
# works almost identically to what we did previously with sciencedata.dk

In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

resp = requests.get("https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD000001-HD010000.zip", headers=headers)
resp

<Response [200]>

In [20]:
resp = requests.get("https://edh-www.adw.uni-heidelberg.de/data/export", headers=headers)
resp

<Response [200]>

In [26]:
import re

In [ ]:
re.match()

In [25]:
resp.text

'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<meta name="keywords" content="latin epigraphy,epigraphy,inscription,database,ancient,Rome,Roman Empire,fotos,images,bibliography" />\n<meta name="description" content="The Epigraphic Database Heidelberg contains the texts of Latin and bilingual (i.e. Latin-Greek) inscriptions of the Roman Empire." />\n<meta name="publisher" content="Heidelberg Academy of Sciences and Humanities" />\n<meta name="author" content="Frank Grieshaber" />\n\n<title>Epigraphic Database Heidelberg</title>\n<link rel="stylesheet" href="/edh-css/jquery-ui-slider-pips.css" />\n<link rel="stylesheet" href="/edh-css/jquery.tree.css" />\n<link rel="stylesheet" href="/edh-css/jq/jquery-ui.css" />\n<link rel="stylesheet" href="https://code.jquery.com/ui/1.10.1/themes/base/jquery-ui.css" />\n<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet/0.7.7/leaflet.css" />\n<l

In [ ]:
url_template = "https://edh-www.adw.uni-heidelberg.de/download/edhEpidocDump_HD000001-HD010000.zip"
for url in urls:
    

In [14]:
zipped = zipfile.ZipFile(io.BytesIO(resp.content))
### names of all files within the zipped directory
namelist = zipped.namelist()[1:] ### index "0" is for main directory
len(namelist)

9929

In [15]:
%%time
edh_xml_data = []
for filename in namelist:
  soup = BeautifulSoup(zipped.read(filename))
  try:
    idno_uri = soup.find("idno", attrs={"type" : "URI"}).get_text()
    idno_tm = soup.find("idno", attrs={"type" : "TM"}).get_text()
    placenames_refs = []
    try: 
      placenames = soup.find_all("placename")
      for placename in placenames:
        placenames_refs.append(placename["ref"])
    except: placenames_refs = []
    text_tag = soup.find("div", attrs={"type" : "edition"})
    commentary = soup.find("div", attrs={"type" : "commentary"}).get_text()
    text = " ".join(text_tag.get_text().splitlines()[1:])
    origdate_text = soup.find("origdate").get_text().replace("\n", "")
    origdate_attrs = soup.origdate.attrs
    objecttype = [soup.find("objecttype").get_text(), str(soup.find("objecttype")).partition("lod/")[2].partition("\">")[0]]
    edh_xml_data.append([idno_uri, idno_tm, placenames_refs, text, origdate_text, origdate_attrs, objecttype, commentary])
  except:
    pass

CPU times: user 51.5 s, sys: 0 ns, total: 51.5 s
Wall time: 51.5 s


In [16]:
edh_xml_data

[['http://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD000002',
  '265631',
  ['http://www.trismegistos.org/place/000172',
   'http://www.trismegistos.org/place/000172',
   'http://www.geonames.org/',
   'http://www.geonames.org/',
   'http://www.geonames.org/countries/it/'],
  'Caius Sextius Paris qui vixit annis LXX             ',
  '51 AD – 200 AD                            ',
  {'notbefore-custom': '0051',
   'notafter-custom': '0200',
   'datingmethod': 'http://en.wikipedia.org/wiki/Julian_calendar'},
  ['Tafel', '257'],
  '\n AE 1983: Breite: 35 cm.\n'],
 ['http://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD000003',
  '220675',
  ['http://www.trismegistos.org/place/025443',
   'http://www.trismegistos.org/place/002036',
   'http://www.geonames.org/2510394',
   'http://www.geonames.org/',
   'http://www.geonames.org/countries/es/'],
  'Publio Mummio Publi filio Galeria Sisennae Rutiliano Xviro stlitibus iudicandis  ',
  '131 AD – 170 AD                            ',
  {'notbefo

In [3]:
### configure session and groupurl
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# EDH_dump.zip from sciencedata.dk

In [7]:
### based on this: https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url
%%time
response = conf[0].get(conf[1] + "SDAM_data/EDH/EDH_dump.zip")
### check the response status
print(response)

<Response [200]>
CPU times: user 576 ms, sys: 425 ms, total: 1 s
Wall time: 37.1 s


In [8]:
%%time
zipped = zipfile.ZipFile(io.BytesIO(response.content))

CPU times: user 1.35 s, sys: 129 ms, total: 1.47 s
Wall time: 1.47 s


In [9]:
### names of all files within the zipped directory
namelist = zipped.namelist()[1:] ### index "0" is for main directory
len(namelist)

157271

In [10]:
### it is too long, there are probably some mac hidden files included as well. 
### Let's remove them
namelist = [name for name in namelist if not "._" in name]
namelist = [name for name in namelist if  ".xml" in name]
len(namelist)

78634

In [12]:
### show first 10 files
namelist[:10]

['EDH_dump/HD046916.xml',
 'EDH_dump/HD078620.xml',
 'EDH_dump/HD000943.xml',
 'EDH_dump/HD011696.xml',
 'EDH_dump/HD031546.xml',
 'EDH_dump/HD077513.xml',
 'EDH_dump/HD029309.xml',
 'EDH_dump/HD001485.xml',
 'EDH_dump/HD036229.xml',
 'EDH_dump/HD068433.xml']

In [18]:
### look how it works in generating soups
list_of_soups = []
for filename in namelist[:10000]:
  soup = BeautifulSoup(zipped.read(filename))
  list_of_soups.append(soup)

In [15]:
### look at one soup
print(list_of_soups[0].prettify)

<bound method Tag.prettify of <?xml version="1.0" encoding="UTF-8"?><?xml-model href="http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng" schematypens="http://relaxng.org/ns/structure/1.0"?><html><body><tei xml:base="ex-epidoctemplate.xml" xml:lang="de" xml:space="preserve" xmlns="http://www.tei-c.org/ns/1.0">
<teiheader>
<filedesc>
<titlestmt>
<title>Grabinschrift auf Tafel</title>
</titlestmt>
<publicationstmt>
<authority>Epigraphische Datenbank Heidelberg</authority>
<idno type="URI">http://edh-www.adw.uni-heidelberg.de/edh/inschrift/HD046916</idno>
<idno type="TM">214091</idno><idno type="localID">HD046916</idno>
<availability>
<p>© Heidelberg Academy of Sciences and Humanities</p>
<licence target="http://creativecommons.org/licenses/by-sa/4.0/">This file is licensed under the Creative Commons Attribution-ShareAlike 4.0 license.
                    </licence>
</availability>
</publicationstmt>
<sourcedesc>
<msdesc>
<msidentifier>
<repository ref="www.trismegistos.org/">Grenobl

# Make soups and extract useful data from them

In [21]:
%%time
edh_xml_data = []
for filename in namelist:
  soup = BeautifulSoup(zipped.read(filename))
  try:
    idno_uri = soup.find("idno", attrs={"type" : "URI"}).get_text()
    idno_tm = soup.find("idno", attrs={"type" : "TM"}).get_text()
    placenames_refs = []
    try: 
      placenames = soup.find_all("placename")
      for placename in placenames:
        placenames_refs.append(placename["ref"])
    except: placenames_refs = []
    text_tag = soup.find("div", attrs={"type" : "edition"})
    commentary = soup.find("div", attrs={"type" : "commentary"}).get_text()
    text = " ".join(text_tag.get_text().splitlines()[1:])
    origdate_text = soup.find("origdate").get_text().replace("\n", "")
    origdate_attrs = soup.origdate.attrs
    objecttype = [soup.find("objecttype").get_text(), str(soup.find("objecttype")).partition("lod/")[2].partition("\">")[0]]
    edh_xml_data.append([idno_uri, idno_tm, placenames_refs, text, origdate_text, origdate_attrs, objecttype, commentary])
  except:
    pass

CPU times: user 8min 48s, sys: 825 ms, total: 8min 49s
Wall time: 8min 49s


In [22]:
%%time
edh_xml_data_df = pd.DataFrame(edh_xml_data, columns=["idno_uri", "idno_tm", "placenames_refs", "text", "origdate_text", "origdate_attrs","objecttype", "commentary"])
edh_xml_data_df.head(5)

CPU times: user 189 ms, sys: 6 ms, total: 195 ms
Wall time: 194 ms


In [23]:
len(edh_xml_data_df) # 78631

78631

In [27]:
### put your dataframe data into this folder
sddk.write_file("SDAM_data/EDH/edh_xml_data_2020-06-23.json", edh_xml_data_df, conf)
# original
# s.put(sciencedata_groupurl + "SDAM_data/EDH/edh_xml_data_df.json", data=edh_xml_data_df.to_json())

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/EDH/edh_xml_data_2020-06-23.json"


In [28]:
import sddk

In [31]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/



# Playground with lists&dicts

In [32]:
df = sddk.read_file("SDAM_data/EDH/edh_xml_data_2020-06-23.json", "df", conf)
df.head(5)

,idno_uri,idno_tm,placenames_refs,text,origdate_text,origdate_attrs,objecttype,commentary
0,http://edh-www.adw.uni-heidelberg.de/edh/insch...,214091,"[http://www.trismegistos.org/place/019860, htt...",In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"{'notbefore-custom': '0551', 'notafter-custom'...","[Tafel, 257]",\n\n
1,http://edh-www.adw.uni-heidelberg.de/edh/insch...,415668,"[http://www.trismegistos.org/place/016530, htt...",Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"{'notbefore-custom': '0071', 'notafter-custom'...","[Tafel, 257]",\n\n
2,http://edh-www.adw.uni-heidelberg.de/edh/insch...,192497,"[http://www.trismegistos.org/place/029481, htt...",Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"{'notbefore-custom': '0197', 'datingmethod': '...","[unbestimmt, 2]",\n Textwiedergabe nach Eck.\n
3,http://edh-www.adw.uni-heidelberg.de/edh/insch...,199108,"[http://www.trismegistos.org/place/015771, htt...",Saturno Meddensi Augusto sacru...,,{'datingmethod': 'http://en.wikipedia.org/wiki...,"[unbestimmt, 2]",\n\n
4,http://edh-www.adw.uni-heidelberg.de/edh/insch...,201034,"[http://www.trismegistos.org/place/015771, htt...",Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"{'notbefore-custom': '0193', 'datingmethod': '...","[unbestimmt, 2]",\n Datierung: L. Naevius Quadratianus war 193 ...


In [42]:
origdate_list = df["origdate_attrs"].tolist()

In [49]:
new_list = []
for el in origdate_list:
    if "notbefore-custom" in el.keys():
        new_list.append(el["notbefore-custom"])
    else:
        new_list.append("")

#  [el["notbefore-custom"] for el in origdate_list if "notbefore-custom" in el.keys() else: ]

In [50]:
new_list

['0551',
 '0071',
 '0197',
 '',
 '0193',
 '0151',
 '0501',
 '0457',
 '',
 '0001',
 '',
 '0250',
 '0137',
 '0208',
 '0296',
 '0122',
 '0171',
 '0175',
 '',
 '',
 '',
 '0101',
 '',
 '0001',
 '',
 '0069',
 '0071',
 '',
 '0151',
 '',
 '0131',
 '0171',
 '0001',
 '-0027',
 '0151',
 '',
 '',
 '0301',
 '0151',
 '',
 '',
 '0171',
 '0001',
 '0101',
 '0001',
 '',
 '0001',
 '0101',
 '0300',
 '0107',
 '0101',
 '0151',
 '0001',
 '0101',
 '',
 '',
 '0301',
 '',
 '0001',
 '0151',
 '0305',
 '0198',
 '0101',
 '0180',
 '0001',
 '',
 '0238',
 '0201',
 '',
 '',
 '0101',
 '0201',
 '0101',
 '0101',
 '0151',
 '0042',
 '0031',
 '0001',
 '',
 '',
 '0001',
 '',
 '0101',
 '0198',
 '',
 '0001',
 '',
 '0151',
 '',
 '0171',
 '',
 '0150',
 '0101',
 '0198',
 '0051',
 '',
 '0001',
 '',
 '0151',
 '0140',
 '0151',
 '',
 '0071',
 '0151',
 '',
 '0001',
 '0171',
 '',
 '0001',
 '0131',
 '0070',
 '',
 '-0025',
 '0101',
 '0001',
 '',
 '',
 '0101',
 '',
 '0151',
 '0069',
 '',
 '0001',
 '0001',
 '0151',
 '',
 '',
 '',
 '',
 '011

In [38]:
def get_notbefore(row):
    try:
        return row["origdate_attrs"]["notbefore-custom"]
    except:
        return ""

In [40]:
df["newcolumn"] = df.apply(lambda row: get_notbefore(row), axis=1)

In [ ]:
df

In [41]:
df

,idno_uri,idno_tm,placenames_refs,text,origdate_text,origdate_attrs,objecttype,commentary,newcolumn
0,http://edh-www.adw.uni-heidelberg.de/edh/insch...,214091,"[http://www.trismegistos.org/place/019860, htt...",In hoc tumulo meserecordia! Christi requiescet...,551 AD – 600 AD,"{'notbefore-custom': '0551', 'notafter-custom'...","[Tafel, 257]",\n\n,0551
1,http://edh-www.adw.uni-heidelberg.de/edh/insch...,415668,"[http://www.trismegistos.org/place/016530, htt...",Alae Noricorum Iulius Quintus Vagdaevercusti v...,71 AD – 130 AD,"{'notbefore-custom': '0071', 'notafter-custom'...","[Tafel, 257]",\n\n,0071
2,http://edh-www.adw.uni-heidelberg.de/edh/insch...,192497,"[http://www.trismegistos.org/place/029481, htt...",Imperator Caesar Lucius Septimius Severus Pius...,197 AD,"{'notbefore-custom': '0197', 'datingmethod': '...","[unbestimmt, 2]",\n Textwiedergabe nach Eck.\n,0197
3,http://edh-www.adw.uni-heidelberg.de/edh/insch...,199108,"[http://www.trismegistos.org/place/015771, htt...",Saturno Meddensi Augusto sacru...,,{'datingmethod': 'http://en.wikipedia.org/wiki...,"[unbestimmt, 2]",\n\n,
4,http://edh-www.adw.uni-heidelberg.de/edh/insch...,201034,"[http://www.trismegistos.org/place/015771, htt...",Lucio Naevio Quadratiano legato Augusti pro pr...,193 AD,"{'notbefore-custom': '0193', 'datingmethod': '...","[unbestimmt, 2]",\n Datierung: L. Naevius Quadratianus war 193 ...,0193
...,...,...,...,...,...,...,...,...,...
78626,http://edh-www.adw.uni-heidelberg.de/edh/insch...,155523,"[http://www.trismegistos.org/place/003201, htt...",Iovi Optimo Maximo et Matribus Ollototis sive ...,201 AD – 300 AD,"{'notbefore-custom': '0201', 'notafter-custom'...","[Altar, 29]",\n\n,0201
78627,http://edh-www.adw.uni-heidelberg.de/edh/insch...,193669,"[http://www.trismegistos.org/place/029481, htt...",Deo Mercurio sacrum,101 AD – 300 AD,"{'notbefore-custom': '0101', 'notafter-custom'...","[unbestimmt, 2]",\n Verschollen. Z. 1 befand sich auf der coron...,0101
78628,http://edh-www.adw.uni-heidelberg.de/edh/insch...,341941,[],Salviane filius,,{'datingmethod': 'http://en.wikipedia.org/wiki...,"[Urne, 78]",\n\n,
78629,http://edh-www.adw.uni-heidelberg.de/edh/insch...,179782,"[http://www.trismegistos.org/place/004366, htt...",CCI N,107 AD – 275 AD,"{'notbefore-custom': '0107', 'notafter-custom'...","[Stele, 250]",\n Unterer Teil eines Reliefs über der Inschri...,0107


In [33]:
df["placenames_refs"].tolist()[:10]

[['http://www.trismegistos.org/place/019860',
  'http://www.trismegistos.org/place/002036',
  'http://www.geonames.org/2970735',
  'http://www.geonames.org/',
  'http://www.geonames.org/countries/fr/'],
 ['http://www.trismegistos.org/place/016530',
  'http://www.trismegistos.org/place/024596',
  'http://www.geonames.org/2893837',
  'http://www.geonames.org/',
  'http://www.geonames.org/countries/de/'],
 ['http://www.trismegistos.org/place/029481',
  'http://www.trismegistos.org/place/029717',
  'http://www.geonames.org/785842',
  'http://www.geonames.org/',
  'http://www.geonames.org/countries/mk/'],
 ['http://www.trismegistos.org/place/015771',
  'http://www.trismegistos.org/place/017214',
  'http://www.geonames.org/',
  'http://www.geonames.org/',
  'http://www.geonames.org/countries/tn/'],
 ['http://www.trismegistos.org/place/015771',
  'http://www.trismegistos.org/place/017336',
  'http://www.geonames.org/',
  'http://www.geonames.org/',
  'http://www.geonames.org/countries/tn/'],
